In [1]:
import proposal as pp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14

# Getting started with PROPOSAL: Propagator

The `Propagator` class provides a full three-dimensional particle simulation.
Given the information about the *propagation environment* and the *initial particle state*, all particle interactions are simulated. 
The *output* of the Propagation provides all information about the propagation process, including the final particle state, a list of interactions during the propagation as well as other methods to get the information that are relevant for you.

In this notebook, we will go through all steps that are necessary so that you can propagate your first particles with PROPOSAL!

## 1. Defining the propagation environment

Before we can propagate the first particle, we need to define the world that our particle lives in.
This includes information like: "Out of what materials is our world made of?", "What parametrizations should be used to describe the physics in our world", "How exact should our propagation be?" and others.

The easiest way to provide this information to PROPOSAL is by using a json *configuration file*.

To get started, we may use one of the configuration files that are given in the `examples` folder of the GitHub repository of PROPOSAL. Let's use the [`config_minimal.json`](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/examples/config_minimal.json), which provides a minimal version of a working configuration file. It looks like this:

```json
{
	"global":
	{
		"cuts":
		{
			"e_cut": 500,
			"v_cut": 1,
			"cont_rand": false
		}
	},
	"sectors": [
		{
			"medium": "ice",
			"geometries": [
				{
					"hierarchy": 0,
					"shape": "sphere",
					"origin": [0, 0, 0],
					"outer_radius": 1e20
				}
			]
		}
	]
}
```

Firstly, we see a `cuts` object. We can use the settings provided here to define the precision of our propagation. For example, a `e_cut` setting of 500 means that only energy losses bigger than 500 MeV are sampled individually. 
If you want to learn more about the concept of `EnergyCuts` to steer the propagation precision and performance, you should take a look at the [`EnergyCut` jupyter notebook](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/examples/EnergyCut.ipynb).

Secondly, we see a section called `sectors`. The propagation environment can be made out of several sectors, where each sector can have different properties.
In this case, we have only one sector: This sector is a sphere with a radius of 1e20 cm and is entirely made of ice.

Of course, there are a lot of other options that you may set with the configuration file.
If you want to learn which options you can set, look at [`docs/config_docu.md`](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/docs/config_docu.md).
There are also other examples of configuration files where more options are set, namely [`config_earth.json`](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/examples/config_earth.json) and [`config_full.json`](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/examples/config_full.json), that you may look at for inspiration.

For now, let us continue with our minimal example configuration file.

## 2. Initializing our Propagator

To create our Propagator object, we can use the following constructor:

```
    pp.Propagator(particle_def, path_to_config_file)
```

The first argument is a `particle_def` object, which includes the static information (like mass, charge or lifetime) about the particle that we want to propagate (note that the dynamic particle properties like energy, position or direction will be defined later!).

The second argument must be a path to a valid json configuration file.

(**Side note:** There is also a different way to define a Propagator that does not use a configuration file. This advanced method is described in the jupyter notebook [`AdvancedPropagator`](https://github.com/tudo-astroparticlephysics/PROPOSAL/blob/master/examples/AdvancedPropagator.ipynb).)

For this example, let's say we want to create a Propagator for a muon using the `config_minimal.json` configuration file:

(**Note:** Creating a Propagator object for a specific configuration for the first time may take some minutes since the interpolation tables have to be created.)

In [2]:
particle = pp.particle.MuPlusDef()
propagator = pp.Propagator(particle, "config_minimal.json")

## 3. Defining the initial particle

We have already decided that we want to propagate a muon.
However, we still need to define the initial state of our particle.

For this, we need to define a `ParticleState` object:

In [3]:
initial_state = pp.particle.ParticleState()
initial_state.energy = 1e6 # energy in MeV
initial_state.position = pp.Cartesian3D(0, 0, 0) # set our particle to the origin of the coordinate system
initial_state.direction = pp.Cartesian3D(0, 0, 1) # let our particle propagate to the z-direction

## 4. Propagating our particle

We have our Propagator object and our initial particle state now, so we have everything that we need to start propagating! 
For this, we use the `propagate` method of our Propagator.

But first, let's have a closer look at the `propagate` method itself:

```
    pp.Propagator.propagate(initial_particle, max_distance=1e20, min_energy=0.0, hierarchy_condition=0)
```
The first (and mandatory) argument is our `ParticleState` object, which defines the initial state of our particle.
All other arguments can be used to define a termination condition, i.e. a condition where our particle propagation should stop:

- The `max_distance` parameter detemines that our particle propagation should stop at the latest when our particle has been propagated for a given distance (in cm). Per default, this is set to 1e20cm.
- The `min_energy` parameter determines that our particle propagation should stop at the latest when our particle has reached this energy (in MeV). Per default, this is set to 0 MeV.
- The `hierarchy_condition` parameter determines that the propagation should stop when we leave a sector with a hierarchy higher than the given `hierarchy_condition` **and** when we enter a sector with a hierarchy lower than the given `hierarchy_condition`. This parameter can therefore be used to tell the Propagator to stop when we leave/enter a certain geometry.

For this example, let's just say that we want to propagate our muon until is has decayed.
In this case, we do not need to specify any conditions:

In [4]:
secondaries = propagator.propagate(initial_state)

## 5. Work with the output

All information about the propagation process is saved in a `Secondaries` object, which is returned by the `propagate` function.
The `Secondaries` object provides a lot of useful utilities to get the specific information that is interesting for every specific usecase. 

To give a first insight, we will go through two examples that explain how to extract information out of our `Secondaries` object:

### 5.1 Get information about the final particle state

We can get the `ParticleState` that describes our particle after propagation by using the line

```
    secondaries.final_state()
```

If we are interested in the energy that our particle has after the propagation as well as the time and distance that our particle has been propagated for, we can do the following:

In [5]:
final_state = secondaries.final_state()
final_energy = final_state.energy
final_distance = final_state.propagated_distance
final_time = final_state.time
print(f"Our final state has an energy of {final_energy} MeV.")
print(f"This state has been reached after the particle has been propagated for {final_distance:.0f} cm and for {final_time:.7f} ns.")

Our final state has an energy of 105.6583745 MeV.
This state has been reached after the particle has been propagated for 110433 cm and for 0.0000037 ns.


### 5.2 Get information about all stochastic interactions

If we are interested in all stochastic energy losses of our particle during propagation, we can use the method `stochastic_losses`.
This method returns a list of `StochasticLoss` objects.
Each of these objects describes a single stochastic energy loss be their `energy`, `type`, `position`, etc.

In [6]:
stochastic_losses = secondaries.stochastic_losses()
for loss in stochastic_losses:
    loss_energy = loss.energy # size of our energy loss in MeV
    loss_parent_energy = loss.parent_particle_energy # energy, in MeV, of our propagated particle 
                                                     # just before the stochastic energy loss 
    loss_type = pp.particle.Interaction_Type(loss.type).name # physical type of our energy loss
    print(f"Energy loss of {loss_energy:.0f} MeV of the type {loss_type} at a particle energy of {loss_parent_energy:.0f} MeV.")

Energy loss of 522 MeV of the type ioniz at a particle energy of 999261 MeV.
Energy loss of 653 MeV of the type epair at a particle energy of 993910 MeV.
Energy loss of 629 MeV of the type photonuclear at a particle energy of 993227 MeV.
Energy loss of 501 MeV of the type ioniz at a particle energy of 991281 MeV.
Energy loss of 1606 MeV of the type epair at a particle energy of 989852 MeV.
Energy loss of 6186 MeV of the type epair at a particle energy of 979234 MeV.
Energy loss of 1232 MeV of the type ioniz at a particle energy of 970205 MeV.
Energy loss of 15517 MeV of the type ioniz at a particle energy of 962523 MeV.
Energy loss of 806 MeV of the type epair at a particle energy of 945748 MeV.
Energy loss of 1529 MeV of the type epair at a particle energy of 937591 MeV.
Energy loss of 766 MeV of the type ioniz at a particle energy of 931212 MeV.
Energy loss of 1015 MeV of the type ioniz at a particle energy of 930421 MeV.
Energy loss of 759 MeV of the type epair at a particle energy 

You can find other examples of how to use the Propagator in the [`AdvancedPropagator`](http://localhost:8888/notebooks/examples/AdvancedPropagator.ipynb#Propagating) jupyter notebook!